In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud_module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME: Update with your username and password, and CRUD Python module name

username = "cacurtis"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Class read method must support returning a list object and accept projection JSON input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ returns the '_id' column, which can have an invalid object type of 'ObjectID'
# We remove it here to avoid data table crashes
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME: Add in Grazioso Salvare’s logo
image_filename = '/home/corvinnacurti_snhu/Desktop/GraziosoSalvareLogo.png'  # Ensure the path is correct
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# FIX ME: Include a unique identifier
app.layout = html.Div([
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}),
    html.H1('Grazioso Salvare Dashboard', style={'textAlign': 'center'}),
    html.Div("Developed by Corvinna Curtis - 04.28.2024", style={'textAlign': 'center'}),
    html.Hr(),
    html.Div(
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster/Individual Tracking', 'value': 'Disaster/Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            placeholder="Select a rescue type...",
        ),
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Div(
        id='map-id',
        className='col s12 m6',
        style={'height': '500px'}
    ),
    html.Div(
        id='graph-id',
        className='col s12 m6'
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """Filters the data table with MongoDB queries."""
    try:
        if filter_type in ['Water Rescue', 'Mountain/Wilderness Rescue', 'Disaster/Individual Tracking']:
            filter_map = {
                'Water Rescue': {'breed': {'$in': ['Labrador Retriever', 'Golden Retriever']}},
                'Mountain/Wilderness Rescue': {'breed': {'$in': ['German Shepherd', 'Bernese Mountain Dog']}},
                'Disaster/Individual Tracking': {'breed': {'$in': ['Beagle', 'Bloodhound']}}
            }
            df_filtered = pd.DataFrame.from_records(db.read(filter_map[filter_type]))
        elif filter_type == 'Reset':
            df_filtered = pd.DataFrame.from_records(db.read({}))
        else:
            df_filtered = pd.DataFrame.from_records(db.read({}))
        
        return df_filtered.to_dict('records')
    except Exception as e:
        print(f"Error in update_dashboard: {e}")
        return []

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graph(viewData):
    """Creates a pie chart for the breeds."""
    if viewData is None:
        return None
    dff = pd.DataFrame(viewData)
    return dcc.Graph(
        figure=px.pie(dff, names='breed', title='Distribution of Breeds')
    )

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """Highlights selected columns."""
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, derived_virtual_selected_rows):
    """Updates map with selected data."""
    if not viewData or not derived_virtual_selected_rows:
        return dl.Map(style={'width': '100%', 'height': '100%'}, center=[30.75, -97.48], zoom=10)
    dff = pd.DataFrame(viewData)
    row = derived_virtual_selected_rows[0]
    return dl.Map(style={'width': '100%', 'height': '100%'}, center=[dff.iloc[row, 13], dff.iloc[row, 14]], zoom=10, children=[
        dl.TileLayer(),
        dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
            dl.Tooltip(dff.iloc[row, 4]),
            dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])
        ])
    ])

app.run_server(debug=True)

Dash app running on http://127.0.0.1:10805/
